## λ calculus

[![lambda calculus in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/boazbk/tcscode/blob/master/lambda_calculus.ipynb)
(Different version) 

The λ calculus is discussed in  [Chapter 8: Equivalent Models of Computation](http://introtcs.org/public/lec_07_other_models.html)

In [ ]:
%%html

<style>
.CodeMirror { width: 100vw; }
.prompt  { visibility: visible; width: 100px !important; }
.prompt_container { visibility: visible; width: 100px !important; }
.container {  width: 95% !important; }
.rendered_html { font-size:0.8em; }
.rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td { font-size: 100%;}
td { font-size: 16px }
</style>


This Python notebook provides a way to play with the lamdba calculus and  evaluate lambda expressions of the form `λvar1(exp1) λvar2(exp2) ...`. If you don't know Python you can safely ignore the Ptyhon code and skip below to where we actually talk about the λ calculus itself.

To better fit with python there are two main differences:

* Instead of writing `λvar.exp` we write `λvar(exp)`

* Instead of simply concatenating two expressions `exp1 exp2` we use the `*` operator and write `exp1 * exp2`. We can also use `exp1, exp2` if they are inside a function call or a variable binding parenthesis.

* To reduce an expression `exp`, use `exp.reduce()`

* Since Python does not allow us to override the default `0` and `1` we use `_0` for `λx(y(y))` and `_1` for `λx(y(x))`. 

## Python code (can skip at first read)

If you don't know Python feel free to skip ahead to the part where we play with the $\lambda$ calculus itself.


In [ ]:
# We define an abstract base class Lambdaexp for lambda expressions
# It has the following subclasses:
# Applicableexp: an expression of the form λx.exp
# Combinedexp: an expression of the form (exp,exp')
# Boundvar: an expression corresponding to a bounded variable
# Unboundvar: an expression corresponding to a free variable
#
# The main operations in a Lambdaexp are:
# 1. Replace: given exp,x and exp', obtain the expression exp[x-->exp']
# 2. Reduce: continuously evaluate expressions to obtain a simpler form
# 3. Apply: given exp,exp', if exp is applicable then apply it to exp', otherwise combine the two 
# (we also use the * operator for it)



import operator ,functools

class Lambdaexp:
    """Lambda expressions base class"""
    
    counter = 0
    call_by_name = True  # if False then do normal form evaluation.
    
    def __init__(self):
        self.mykey = {}
    
    def apply(self,other): 
        """Apply expression on an argument"""
        return self*other
        
    def _reduce(self,maxlevel=100): 
        """Reduce expression"""
        return self
    
    def replace(self,old,new): 
        """Replace all occurences of old with new"""
        raise NotImplementedError

    
    def bounded(self): 
        """Set of bounded variables inside expression"""
        return set()

    def asstring(self, m,pretty=False):
        """Represent self as a string mapping bounded variables to particular numbers."""
        raise NotImplementedError

    #------------------------------------------------------------------------------#
    # Ignore this code  in first read: Python specific details

    lambdanames = {}
    reducedstrings = {}
   
    def reduce(self,maxlevel=100):
        if not maxlevel: return self
        #m = {b:b for b in self.bounded() }
        #t = Lambdaexp.reducedstrings.get((self.asstring(m),maxlevel),None)
        #if t: return t
        return self._reduce(maxlevel)
        #k = t.asstring(m)
        #for i in range(maxlevel+1):  
        #    Lambdaexp.reducedstrings[(k,i)] = t
        #return t
            
            
        
    def __mul__(self,other):
        """Use * for combining."""
        return Combinedexp(self,other) if other else self
    
    def __call__(self,*args): 
        """Use function call for application"""
        return functools.reduce(operator.mul,args,self)
    
    def _key(self,maxlevel=100): 
        #if maxlevel not in self.mykey: 
        return self.reduce(maxlevel).__repr__()
            # for i in range(maxlevel+1): self.mykey[i] = s 
        # return self.mykey[maxlevel]

    def __eq__(self,other): return self._key()==other._key() if isinstance(other,Lambdaexp) else False
    def __hash__(self): return hash(self._key())
    
        
        
    def __repr__(self,pretty=False):
        B = sorted(self.bounded())
        m ={}
        for v in B: m[v] = len(m)
        return self.asstring(m,pretty)
    
    
    def _repr_pretty_(self, p, cycle):
        if cycle: p.text( self._repr())
        p.text( self.reduce().__repr__(True))     
        
    def addconst(self,srep):
        """Return either exp.string or replaced with a keyword if it's in table."""
        if self in Lambdaexp.lambdanames:  return blue(Lambdaexp.lambdanames[self])
        return srep

      #------------------------------------------------------------------------------#


    
    
 

In [ ]:
#-------------------------------------------------#
# Utility functions: print color
def bold(s,justify=0):
    return "\x1b[1m"+s.ljust(justify)+"\x1b[21m"

def underline(s,justify=0):
    return "\x1b[4m"+s.ljust(justify)+"\x1b[24m"

def red(s,justify=0):
    return  "\x1b[31m"+s.ljust(justify)+"\x1b[0m"


def green(s,justify=0):
    return  "\x1b[32m"+s.ljust(justify)+"\x1b[0m"


def blue(s,justify=0):
    return  "\x1b[34m"+s.ljust(justify)+"\x1b[0m"
#--------------------------------------------------#

In [ ]:
  
    
    

        
class Applicableexp(Lambdaexp):
    """Lambda expression that can be applied"""
    
    def __init__(self,exp,name):
        Lambdaexp.counter += 1
        self.arg =  Lambdaexp.counter
        self.inner = exp.replace(name,Boundvar(self.arg))
        super().__init__()    
    
    def apply(self,other): 
        return self.inner.replace(self.arg,other)
    
    def replace(self,old,new): 
        if self.arg==old: 
            if not isintance(new,Boundvar): 
                raise TypeError(f"Can't replace {old} in {self} since {new} is not instance of Boundvar")
            self.arg = new.myid
        return Applicableexp(self.inner.replace(old,new),self.arg)
    
    def bounded(self): return self.inner.bounded()|{self.arg}
    
    def _reduce(self,maxlevel=100):
        if Lambdaexp.call_by_name: return self 
        # in call by name there are no reductions inside abstractions
        inner = self.inner.reduce(maxlevel-1)
        return Applicableexp(inner,self.arg)
    
    def asstring(self, m,pretty=False):
        if not pretty: return "λ"+Boundvar(self.arg).asstring(m,False)+".("+self.inner.asstring(m)+")"
        return self.addconst(green("λ")+Boundvar(self.arg).asstring(m,True)+".("+self.inner.asstring(m,True)+")")
        
        
 
  

In [ ]:
class Boundvar(Lambdaexp):
    """Bounded variable"""
    def __init__(self,arg): 
        self.myid = arg
        super().__init__()
        
    def replace(self,argnum,exp): return exp if argnum==self.myid else self
    
    def bounded(self): return { self.myid }

    def asstring(self, m,pretty=False):
        arg = m.get(self.myid,self.myid) 
        return chr(ord('α')+arg)
    

class Unboundvar(Lambdaexp):
    """Unbounded (free) variable."""
    def __init__(self,name): 
        self.name = name
        super().__init__()
        
    def replace(self,name,arg): return arg if name==self.name else self
    
    def asstring(self, m,pretty=False):
        return self.addconst(self.name) if pretty else self.name

   
    
class Combinedexp(Lambdaexp):
    """Combined expression of two expressions."""
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2
        super().__init__()
    
    def replace(self,arg,exp): 
        return Combinedexp(self.exp1.replace(arg,exp),self.exp2.replace(arg,exp))
    
    def bounded(self): return self.exp1.bounded()|self.exp2.bounded()
    
    
    def _reduce(self,maxlevel=100):
        if not maxlevel: return self
        e1 = self.exp1.reduce(maxlevel-1)
        if isinstance(e1,Applicableexp): 
            return  e1.apply(self.exp2).reduce(maxlevel-1)
        return Combinedexp(e1,self.exp2)
    
    def asstring(self, m,pretty=False):
        s = f"({self.exp1.asstring(m,False)} {self.exp2.asstring(m,False)})"
        if not pretty: return s
        return f"({self.exp1.asstring(m,True)} {self.exp2.asstring(m,True)})"


In [ ]:
class λ:
    """Binds a variable name in a lambda expression"""

    def __init__(self,*varlist): 
        """
        Get list of unbounded variables (for example a,b,c) and returns an operator that binds an expresion exp to
        λa(λb(λc(exp))) and so on."""
        if not varlist: raise ValueError("Need to bind at least one variable")
        self.varlist = varlist[::-1]
        
    
    def bindexp(self,exp):
        res = exp
        for v in self.varlist: 
            res = Applicableexp(res,v.name)
        return res
    
    #------------------------------------------------------------------------------#
    # Ignore this code  in first read: Python specific details

    
    def __call__(self,*args):
        exp = functools.reduce(operator.mul,args[1:],args[0])
        return self.bindexp(exp)
    #------------------------------------------------------------------------------#

### Initalization 

The above is all the code for implementing the λ calculus. We now add some convenient global variables: 
λa .... λz and a ... z for variables, and 0 and 1.

In [ ]:
Lambdaexp.lambdanames  = {}
import string

def initids(g):
    """Set up parameters a...z and correpsonding Binder objects λa..λz"""
    lcase = list(string.ascii_lowercase)
    ids = lcase + [n+"_" for n in lcase]
    for name in ids:
        var =  Unboundvar(name)
        g[name] = var
        g["λ"+name] = λ(var)
        Lambdaexp.lambdanames[var] = name

In [ ]:
initids(globals())

In [ ]:
# testing...
λy(y)

In [ ]:
λ(a,b)(a)

In [ ]:
def setconstants(g,consts):
    """Set up constants for easier typing and printing."""
   
    for name in consts:
        Lambdaexp.lambdanames[consts[name]] = name
        if name[0].isalpha(): 
            g[name]=consts[name]
            
        else: # Numeric constants such as 0 and 1 are replaced by _0 and _1
            g["_"+name] = consts[name]
    

def register(g,*args):
    for name in args:
        Lambdaexp.lambdanames[g[name]] = name

In [ ]:
# testing
λa(λz(a))

# λ calculus introduction

* Invented by Church in the 1930s
* Computation by applying functions - pure functions, no side effect
* Inspiration for LISP, ML, OCaml, Haskell, and all functional programming languages
* Extensions of λ calculus crucial to modern programming languages and type systems, proof asistant.

$\lambda x. f$ defines the function $x \mapsto f(x)$. 

__Example:__ $\lambda x. x^2$ defines the function $x \mapsto x^2$. 

Equivalent to $\lambda y. y^2$

In [ ]:
# What will be printed?
F = lambda x: x**2 
F(5)

In [ ]:
# What will be printed?
H = lambda g: g(8)
H(3)

In [ ]:
# what will be printed
H = lambda g: (lambda x: g(g(g(x))) )
H(F)(5)

$\lambda x. exp$ defines a function that maps any argument $z$ to $exp[x \rightarrow z]$

We sometimes use `λx(exp)` instead of $\lambda x.exp$ below

* Every $\lambda$ expression defines a function. $\lambda x.exp$ is the function $z \mapsto exp[x\rightarrow z]$

* Can simulate functions with many arguments by "currying"

In [ ]:
add = lambda a: (lambda b: a+b)
add(5)(8)

__Shorthand:__ $\lambda a,b. exp$ is $\lambda x. (\lambda b.exp)$

* $\lambda$ expressions can be inputs to other $\lambda$ expressions

In [ ]:
(lambda f: lambda x: f(f(x))) (lambda x: x+2) (7)

__Conventions:__ To reduce parenthesis, use $f x$ for $f(x)$. 

Evaluate from left so $f a b = (f a) b$. 

In Python we use `f(a)` or `f*a` for $f a$

###  λ calculus is Turing Complete 

(A 42 page proof on [woodrush.github.io/blog/lambdalisp.html](https://woodrush.github.io/blog/lambdalisp.html) )

![](https://i.imgur.com/xowmMIt.png)

<center><font size="+3"><b>...</b></font></center> 

![](https://i.imgur.com/EVD22lz.png)

# True/False or 1/0 in $\lambda$ calculus

In [ ]:
T = λa(λb(a))
F = λa(λb(b))

In [ ]:
F(x,y)

In [ ]:
T(x,y)

In [ ]:
IF = λ(a,b,c)( (a(b))(c) )

In [ ]:
# What will be printed?
IF(T,x,y)

In [ ]:
# can also write it as
IF =  λ(a,b,c)(a*b*c)
IF(F,x,y)

We define $1 = \lambda a,b.a$ and $0 = \lambda a,b.b$. We define $IF = \lambda a,b,c. a b c$ 

In [ ]:
_1 = T
_0 = F

In [ ]:
setconstants(globals(),{"1" : λ(x,y)(x) , "0" : λ(x,y)(y)  })

In [ ]:
λa(λb(b))==_0

__Question:__ Suppose that $F = \lambda f. (\lambda x. (f x)f)$, $1 = \lambda x.(\lambda y.x)$ and $0=\lambda x.(\lambda y.y)$.
What is $F \; 1\; 0 = (F 1) 0$?

__a.__ $1$

__b.__ $0$

__c.__ $\lambda x.1$

__d.__ $\lambda x.0$

In [ ]:
F= λf( λx( f(x)(f) ))

In [ ]:
F(_1)

In [ ]:
F(_1)==λx(_1(x,_1))

In [ ]:
_1(_0,_1)

In [ ]:
F(_1)(_0)

# Reminders: 

$\lambda x. exp$ corresponds to the function that maps any argument $z$ to the expression $exp[x\rightarrow z]$. 

We use "Currying" for two argument functions, and so $\lambda a,b. f(a,b)$ is the same as $\lambda a. (\lambda b.(f(a))(b) )$. We drop parenthesis for function application and evaluate from the left and so also write the latter as $\lambda a. (\lambda b. f\; a\;b)$

We define $1 = \lambda a,b.a$ and $0=\lambda a,b.b$

### Some useful functions

Let us now add some of the basic functions in the λ calculus

In [92]:
PAIR =λx(λy(λf(f(x,y)))) # given x,y returns the function f --> f(x,y)
HEAD = λp(p(_1) )
TAIL  =λp(p(_0) )

__Lemma:__ If $P = \text{PAIR}\; a\; b$ then $\text{HEAD}\; P = a$ and $\text{TAIL}\; P\;= b$

__Proof:__ $\text{PAIR}\; a\; b = \lambda f.f(a,b)$ 

The function mapping $f$ to $f(a,b)$

Hence $P 1 = a$ and $P 0 = b$

Hence $(\lambda p. p 1) P = a$ and $(\lambda p. p 0) P = b$  ▮

In [93]:
HEAD(PAIR(x,y))

x

In [94]:
# What will be printed?
L = PAIR(x, PAIR(y, z))
HEAD( TAIL( L ))

y

## Lists
We can construct list by repeatedly using `PAIR`. We define a special term `NIL` for the empty list. If $L$ is the list $[b,c,d]$ then $PAIR(a,L)$ is the list $[a,b,c,d]$

In [ ]:
def slist(L, maxrec=100):
    if maxrec<0: return red("TOO LONG")
    if ISEMPTY(L)==_1: return red("NIL")
    return HEAD(L).reduce().__repr__(True)+", "+slist(TAIL(L),maxrec-1)
def plist(L): print("["+slist(L)+"]")

In [95]:
NIL= λf(_1)
ISEMPTY= λp(p(λx(λy(_0))))
ISEMPTY(NIL)

1

In [ ]:
register(globals(),"NIL", "PAIR")

In [96]:
ISEMPTY(PAIR(_1,_1))

0

In [97]:
L = PAIR(x, PAIR(y,NIL))
L

λα.(((α x) ((PAIR y) NIL)))

In [98]:
plist(L)

[x, y, NIL]


We can make lists of bits as follows:

In [99]:
def makelist(*L):
    """Construct a λ list of _0's and _1's."""
    if not L: return NIL
    h = _1 if L[0]   else _0
    return PAIR(h,makelist(*L[1:]))

In [105]:
L=makelist(1,0,1,1,0)
plist(L)
#L.__repr__()

[1, 0, 1, 1, 0, NIL]


In [106]:
plist(TAIL(L))

[0, 1, 1, 0, NIL]


In [107]:
HEAD(TAIL(L))

0

## Recursion

We now show how we can implement recursion in the λ calculus. We start by doing this in Python.
Let's try to define XOR in a recursive way and then avoid recursion

In [108]:
# XOR of 2 bits
def xor2(a,b): return 1-b if a else b

# XOR of a list - recursive definition
def xor(L): return xor2(L[0],xor(L[1:])) if L else 0

xor([1,0,0,1,1])

1

### Making this non recursive 

In [109]:
def myxor(me,L): return 0 if not L else xor2(L[0],me(L[1:]))

__First attempt:__ `xor(L)= myxor(myxor,L)` 

In [110]:
def xor(L): return myxor(myxor,L)
xor([])

0

In [111]:
xor([0,1,1])

TypeError: myxor() missing 1 required positional argument: 'L'

The issue is that `myxor` takes _two_ arguments, while in `me` we only supply one.
Thus, we will modify `myxor` to `tempxor` where we replace the call `me(x)` with `me(me,x)`:

__Second attempt:__ Recall
```python
def myxor(me,L): return 0 if not L else xor2(L[0],me(L[1:]))
```

In [112]:
def tempxor(temp,L): return myxor(lambda x: temp(temp,x),L)

__Sanity check:__ `temp` takes _two_ arguments, but `myxor` is called with a function that takes _one_ argument `x` and maps it to `temp(temp,x)`

In [113]:
def xor(L): return tempxor(tempxor,L)

xor([1,0,1,1])

1

```python
def myxor(me,L): return 0 if not L else xor2(L[0],me(L[1:]))

def tempxor(temp,L): return myxor(lambda x: temp(temp,x),L)

```
__Claim:__ For every list `L`, `tempxor(tempxor,L)=XOR(L)`.

__Base:__ Trivial

__Inductive step:__ If `myxor` gets the function `x ↦ tempxor(tempxor,x)` and applies it to `L[1:]` then by induction this is `XOR(L[:1])`

### Generalization 

The `RECURSE` operator will take a function `f` that takes two arguments `me` and `x` and return a function `g`  where the calls to  `me` are replaced with calls to `g`

In [117]:
def fact_nr(me,n): return 1 if n==0 else me(n-1)*n

def RECURSE(f):
    def ftemp(me,x): return f(lambda x: me(me,x),x)
    return lambda x: ftemp(ftemp,x)

fact = RECURSE(fact_nr)

fact(4)

24

### The λ version

We now repeat the same arguments with the λ calculus:

In [118]:
# XOR of two bits
XOR2 = λ(a,b)(IF(a,IF(b,_0,_1),b))

# Recursive XOR with recursive calls replaced by m parameter
myXOR = λ(m,l)(IF(ISEMPTY(l),_0,XOR2(HEAD(l),m(TAIL(l)))))

Recall Python RECURSE:
```python
def RECURSE(f):
    def ftemp(me,x): return f(lambda x: me(me,x),x)
    return lambda x: ftemp(ftemp,x)
```

In [119]:
# Recurse operator (aka Y combinator)
# ftemp(me) = λx( f(me(me))(x)  ) = f(me(me))
# ftemp(ftemp,x) = ftemp(ftemp)(x)  
RECURSE = λf(  λm(f(m(m)))   (λm(f(m(m)))) )
# XOR function
XOR = RECURSE(myXOR)

Let's test this out:

In [122]:
XOR(PAIR(_0,NIL)) # List [0]

0

In [124]:
XOR(PAIR(_1,PAIR(_1,PAIR(_1,NIL)))) # List [1,1,1]

1

In [129]:
XOR(makelist(1,0,1,1,1,1))

1

In [ ]:
XOR(makelist(1,0,0,1,1))

### Knights of the Lambda Calculus emblem 

![](https://i.imgur.com/1I1XL9f.png)

![](https://i.imgur.com/VY4Yzc7.jpg)

```python
RECURSE = λf(  λm(f(m(m)))   (λm(f(m(m)))) )
```